In [1]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
mask = np.load(r"C:\Users\Research\aman fastmri\Data\mask_4x_320_random.npy")  # Shape: (1, 320, 320)
print("og shape:", mask.shape)

# # Use np.tile to reshape it to (1, 320, 320, 1)
# # var_sampling_mask = np.tile(var_sampling_mask[..., np.newaxis], (1, 1, 1, 1))  # Final shape: (1, 320, 320, 1)
# mask = np.tile(mask, (1, 320, 1, 2))  # tile height=320 times

# # Confirm final shape
# print("New shape:", mask.shape) 
# mask_for_plot = np.squeeze(mask[...,0])  # Shape: (320, 320)

# # Plot
# plt.figure(figsize=(5, 5))
# plt.imshow(mask_for_plot, cmap='gray')
# plt.title("Tiled Sampling Mask (320x320)")
# plt.axis('off')
# plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Research\\aman fastmri\\Data\\mask_4x_320_random.npy'

In [3]:
import h5py
import numpy as np
import tensorflow as tf

def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

class MRISliceGeneratorMag(tf.keras.utils.Sequence):
    """
    Data generator for magnitude-only MRI reconstruction.

    Input  : undersampled magnitude image  (B, H, W, 1)
    Target : fully-sampled magnitude image (B, H, W, 1)
    """

    def __init__(self, file_list, batch_size=4, shuffle=True):
        self.file_list = file_list
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.slice_index_map = []
        self._build_index()

    def _build_index(self):
        for file_idx, file_path in enumerate(self.file_list):
            with h5py.File(file_path, 'r') as f:
                num_slices = f['image_under'].shape[0]
                for slice_idx in range(num_slices):
                    self.slice_index_map.append((file_idx, slice_idx))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.slice_index_map) / self.batch_size))

    def __getitem__(self, index):
        batch_map = self.slice_index_map[
            index * self.batch_size : (index + 1) * self.batch_size
        ]

        input_mag_batch = []
        target_mag_batch = []

        for file_idx, slice_idx in batch_map:
            with h5py.File(self.file_list[file_idx], 'r') as f:
                img_under = f['image_under'][slice_idx]  # (H, W, 2)
                img_full  = f['image_full'][slice_idx]   # (H, W, 2)

                # Convert to complex
                img_under_c = to_complex(img_under)
                img_full_c  = to_complex(img_full)

                # Magnitude
                img_under_mag = np.abs(img_under_c)
                img_full_mag  = np.abs(img_full_c)

                input_mag_batch.append(img_under_mag)
                target_mag_batch.append(img_full_mag)

        # Stack and add channel dimension
        x_batch = np.stack(input_mag_batch, axis=0)[..., np.newaxis]  # (B, H, W, 1)
        y_batch = np.stack(target_mag_batch, axis=0)[..., np.newaxis] # (B, H, W, 1)

        return x_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.slice_index_map)


In [4]:
val_folder = r"E:\fastmri_singlecoil_FSSCAN\val_norm"
import h5py
import numpy as np
import glob
import os

kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))

half_val = len(kspace_files_list_val) 
half_val

199

In [5]:
import h5py
import numpy as np
import glob
import os

kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))


half_val = len(kspace_files_list_val) 

kspace_files_list_val = kspace_files_list_val[:half_val]


val_gen = MRISliceGeneratorMag(kspace_files_list_val, batch_size=1, shuffle=False)


print(len(val_gen))  


7135


In [3]:
%run ./DCRCNN.ipynb


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# ============================================================
# Directory Setup
# ============================================================
save_dir = "./SavedModels_DCRCNN_full_2"
H, W = 320, 320
EPOCHS = 50
LEARNING_RATE = 1e-4

In [5]:
model = build_dcr_cnn(
    input_shape=(320, 320, 2),
    num_dcr_blocks=10,   # or 3 / 8
    num_features=64,
    growth_rate=32
)
import numpy as np

def count_parameters_millions(model):
    trainable = np.sum([np.prod(v.shape) for v in model.trainable_variables])
    non_trainable = np.sum([np.prod(v.shape) for v in model.non_trainable_variables])
    total = trainable + non_trainable
    return (
        total / 1e6,
        trainable / 1e6,
        non_trainable / 1e6
    )

total_M, trainable_M, non_trainable_M = count_parameters_millions(model)

print("\n" + "=" * 40)
print(f"Total parameters:       {total_M:.3f} M")
print(f"Trainable parameters:   {trainable_M:.3f} M")
print(f"Non-trainable params:   {non_trainable_M:.3f} M")
print("=" * 40)



Total parameters:       1.202 M
Trainable parameters:   1.202 M
Non-trainable params:   0.000 M


In [6]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

def compute_flops(model, input_shape):
    """
    input_shape: tuple, e.g. (1, H, W, 2)
    returns FLOPs (float operations) for one forward pass
    """

    @tf.function
    def forward(x):
        return model(x)

    concrete_func = forward.get_concrete_function(
        tf.TensorSpec(input_shape, tf.float32)
    )

    frozen_func = convert_variables_to_constants_v2(concrete_func)
    graph_def = frozen_func.graph.as_graph_def()

    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name="")

        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

        flops = tf.compat.v1.profiler.profile(
            graph=graph,
            run_meta=run_meta,
            cmd="op",
            options=opts
        )

    return flops.total_float_ops


In [7]:
# Example: infer H, W from your data or define explicitly
H, W = 320,320

flops = compute_flops(model, input_shape=(1, H, W, 2))

print(f"FLOPs (single forward pass): {flops / 1e9:.2f} GFLOPs")


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
FLOPs (single forward pass): 245.98 GFLOPs


In [8]:


# ============================================================
# Optimizer & Compile
# ============================================================
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss="mse")

# ============================================================
# Load Initial Weights (Optional Resume)
# ============================================================
if tf.train.latest_checkpoint(save_dir):
    model.load_weights(tf.train.latest_checkpoint(save_dir))
    print("✅ Loaded latest checkpoint")
else:
    print("ℹ️ No checkpoint found. Training from scratch.")

✅ Loaded latest checkpoint


In [9]:
import os
import numpy as np
import h5py
import glob
import pandas as pd
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# --------------------------------------------------
# LOAD MODEL WEIGHTS
# --------------------------------------------------
#model.load_weights("./weight_sfu_fastmri_complex_perploss_fsa.h5")

# --------------------------------------------------
# HELPERS
# --------------------------------------------------
def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt,
        pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03,
        full=False
    )

# --------------------------------------------------
# STORAGE (FOR EXCEL)
# --------------------------------------------------
volume_metrics = []   # volume-wise PSNR, NMSE
slice_metrics  = []   # slice-wise SSIM
ssim_list = []
psnr_list = []
nmse_list = []

# --------------------------------------------------
# PROCESSING
# --------------------------------------------------
for file in tqdm(kspace_files_list_val, desc="Processing volumes"):
    volume_name = os.path.basename(file)

    with h5py.File(file, 'r') as f:
        image_full  = f["image_full"][:]    # (S, H, W, 2)
        image_under = f["image_under"][:]   # (S, H, W, 2)
        max_val     = float(f["max_val_full_image"][0])

    # --------------------------------------------------
    # Model inference
    # --------------------------------------------------
    pred = model.predict(image_under, verbose=0)  # (S, H, W, 2)

    # --------------------------------------------------
    # Rescale
    # --------------------------------------------------
    image_full = image_full * max_val
    pred       = pred * max_val

    # --------------------------------------------------
    # Magnitude images
    # --------------------------------------------------
    gt_mag   = np.abs(to_complex(image_full))
    pred_mag = np.abs(to_complex(pred))

    # --------------------------------------------------
    # pred *= max_val  # Scale predicted output to original intensity range
    psnr_val = peak_signal_noise_ratio(gt_mag, pred_mag, data_range=max_val)

    # Volume-wise PSNR and NMSE
    #psnr_val = peak_signal_noise_ratio(gt_mag,pred_mag, data_range=max_val)
    nmse_val = nmse(gt_mag.flatten(), pred_mag.flatten())

    psnr_list.append(psnr_val)
    nmse_list.append(nmse_val)

    volume_metrics.append({
        "volume_name": volume_name,
        "num_slices": gt_mag.shape[0],
        "PSNR_dB": psnr_val,
        "NMSE": nmse_val
    })

    # --------------------------------------------------
    # Slice-wise SSIM (2D per slice)
    # --------------------------------------------------
    for i in range(gt_mag.shape[0]):
        ssim_val = compute_ssim(
            gt_mag[i],        # (H, W)
            pred_mag[i],      # (H, W)
            max_val
        )

        slice_metrics.append({
            "volume_name": volume_name,
            "slice_index": i,
            "SSIM": ssim_val
        })

# --------------------------------------------------
# SAVE TO EXCEL
# --------------------------------------------------
df_volume = pd.DataFrame(volume_metrics)
df_slice  = pd.DataFrame(slice_metrics)

output_excel = "evaluation_metrics_DCRCNN.xlsx"

with pd.ExcelWriter(output_excel, engine="openpyxl") as writer:
    df_volume.to_excel(writer, sheet_name="Volume_Metrics", index=False)
    df_slice.to_excel(writer, sheet_name="Slice_SSIM", index=False)

# --------------------------------------------------
# REPORT
# --------------------------------------------------
print("\n" + "=" * 50)
print(f"Excel file saved to: {output_excel}")
print("=" * 50)

print(f"PSNR (Mag, volume): {df_volume['PSNR_dB'].mean():.4f} ± {df_volume['PSNR_dB'].std():.4f} dB")
print(f"NMSE (Mag, volume): {df_volume['NMSE'].mean():.6f} ± {df_volume['NMSE'].std():.6f}")
print(f"SSIM (Mag, slice):  {df_slice['SSIM'].mean():.4f} ± {df_slice['SSIM'].std():.4f}")
print("=" * 50)


Processing volumes: 100%|████████████████████████████████████████████████████████████| 199/199 [07:56<00:00,  2.39s/it]



Excel file saved to: evaluation_metrics_DCRCNN.xlsx
PSNR (Mag, volume): 32.6638 ± 2.8763 dB
NMSE (Mag, volume): 0.027041 ± 0.017342
SSIM (Mag, slice):  0.7634 ± 0.0853


In [10]:
import os
import numpy as np
import h5py
import glob
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# # Path to validation folder
# # val_folder = "F:/denoised_preprocessed_h5_val"

# val_folder = r"E:\fastmri\val_norm"
# # val_folder = r"D:\val_norm"

# # val_folder = r"G:\val_norm\val_norm"
# # files = sorted([os.path.join(val_folder, f) for f in os.listdir(val_folder) if f.endswith(".h5")])
# kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))
file_paths = kspace_files_list_val


# ----------------------
# HELPERS
# ----------------------
def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2 + 1e-10)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt, pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03
    )

# ----------------------
# STORAGE
# ----------------------
ssim_list = []
psnr_list = []
nmse_list = []

# ----------------------
# PROCESSING
# ----------------------
for file in tqdm(kspace_files_list_val, desc="Processing volumes"):
    with h5py.File(file, 'r') as f:
        image_under = f["image_under"][:]   # (S, H, W, 2)
        image_full   = f["image_full"][:]     # (S, H, W, 2)
        max_val = float(f["max_val_full_image"][0])

    num_slices = image_under.shape[0]
        # Convert to complex
    # img_under_c = to_complex(image_under)
    # img_full_c  = to_complex(image_full)

    # # Magnitude
    # img_under_mag = np.abs(img_under_c)
    # img_full_mag  = np.abs(img_full_c)
    # img_full_mag = np.expand_dims(img_full_mag, axis=-1)


    # --------------------------------------------------
    # TILE MASK FOR THIS VOLUME
    # --------------------------------------------------
    #mask_batch = np.tile(mask, (num_slices, 1, 1, 1))

    # --------------------------------------------------
    # MODEL INFERENCE
    # --------------------------------------------------
    pred = model.predict(
        image_under,
        batch_size=1,
        verbose=0
    )
    #print("pred",pred.shape)
    #print("img_full_mag",img_full_mag.shape)
    

    #max_val=img_full_mag.max()
    image_full *= max_val
    
    pred *= max_val  # Scale predicted output to original intensity range
    

    # # Convert to complex and get magnitude
    gt_mag = np.abs(to_complex(image_full))
    pred_mag = np.abs(to_complex(pred))

    # Volume-wise PSNR and NMSE
    #psnr_val = peak_signal_noise_ratio(gt_mag, pred_mag, data_range=max_val)
    nmse_val = nmse(gt_mag.flatten(), pred_mag.flatten())
    psnr_val = peak_signal_noise_ratio(gt_mag, pred_mag, data_range=max_val)

    psnr_list.append(psnr_val)
    nmse_list.append(nmse_val)

    # Slice-wise SSIM
    for i in range(gt_mag.shape[0]):
        ssim_val = compute_ssim(gt_mag[i], pred_mag[i], max_val)
        ssim_list.append(ssim_val)
# ----------------------
# REPORT
# ----------------------
print("\n" + "=" * 40)
print(f"PSNR : {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f} dB")
print(f"NMSE (Mag, volume): {np.mean(nmse_list):.6f} ± {np.std(nmse_list):.6f}")
print(f"SSIM (Mag, slice):  {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")

print("=" * 40)


Processing volumes: 100%|████████████████████████████████████████████████████████████| 199/199 [06:36<00:00,  1.99s/it]


PSNR : 32.6637 ± 2.8691 dB
NMSE (Mag, volume): 0.027042 ± 0.017300
SSIM (Mag, slice):  0.7634 ± 0.0853
